**Extracting Dataset**

In [1]:
import zipfile

# Path to the zip file and extract destination
zip_file_path = '/content/archive.zip'
extract_to = '/content/dataset/'

# Unzipping
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Unzipping completed.")

Unzipping completed.


**Installing Libraries:**

In [2]:
import os
import numpy as np
import pandas as pd
import cv2
# from google.colab.patches import cv2_imshow
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [3]:
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Dense, Flatten, Dropout,BatchNormalization ,Activation, InputLayer
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [18]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [4]:
df=pd.read_csv('/content/dataset/labels.csv')

In [5]:
df=df.loc[df['label']!='contempt']
df=df.loc[df['label']!='happy']
df=df.loc[df['label']!='surprise']
df = df.reset_index(drop=True)
df.shape

(15979, 4)

In [11]:
df.head()

,Unnamed: 0,pth,label,relFCs
0,1,anger/image0000060.jpg,anger,0.852311
1,2,anger/image0000061.jpg,anger,0.800957
2,3,anger/image0000066.jpg,disgust,0.843079
3,4,anger/image0000106.jpg,anger,0.849108
4,5,anger/image0000132.jpg,anger,0.819448


In [12]:
df['label'].value_counts()

,count
label,
anger,3608
disgust,3472
fear,3043
sad,2995
neutral,2861


In [ ]:
x=[]
y=[]
emotions = ['surprise', 'happy', 'anger', 'disgust', 'fear', 'sad', 'neutral']

data_dir='/content/dataset/'

for df_index in range(len(df)):
    img_path=os.path.join(data_dir, df['pth'][df_index])
    img=cv2.imread(img_path)
    image_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(image_gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    if len(faces) > 0:
        # If faces are detected, crop the first face
        x1, y1, w, h = faces[0]
        face_crop_img = img[y1:y1+h, x1:x1+w]
        face_crop_img = cv2.cvtColor(face_crop_img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(face_crop_img, (96, 96))
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (96, 96))


    x.append(img)
    y.append(emotions.index(df['label'][df_index]))

x=np.array(x)
y=np.array(y)

In [9]:
x=x/255.0
y = to_categorical(y, num_classes=len(emotions))

In [ ]:
def plot_img(i):
    plt.imshow(x[i])
    plt.axis('off')
    plt.show()
    print(df['label'][i])
    print(y[i])
plot_img(3292)

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=2)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=2)

In [19]:
print(x_train.shape, x_val.shape, x_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

(12223, 96, 96, 3) (1359, 96, 96, 3) (2397, 96, 96, 3)
(12223, 5) (1359, 5) (2397, 5)


In [ ]:
# Load ResNet50 without the top layer
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(96, 96, 3))

# Freeze all layers in the base ResNet50 model
for layer in base_model.layers[:-50]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(5, activation='softmax')(x)  # Adjust number of classes as needed

model = Model(inputs=base_model.input, outputs=output)

In [22]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 96, 96, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 102, 102, 3)    │              0 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 48, 48, 64)     │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 50, 50, 64)     │              0 │ conv1_conv[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 24, 24, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_preact_bn    │ (None, 24, 24, 64)     │            256 │ pool1_pool[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_preact_relu  │ (None, 24, 24, 64)     │              0 │ conv2_block1_preact_b… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 24, 24, 64)     │          4,096 │ conv2_block1_preact_r… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 24, 24, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 24, 24, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_pad        │ (None, 26, 26, 64)     │              0 │ conv2_block1_1_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 24, 24, 64)     │         36,864 │ conv2_block1_2_pad[0]… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 24, 24, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 24, 24, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv  

 Total params: 23,575,045 (89.93 MB)

 Trainable params: 16,362,501 (62.42 MB)

 Non-trainable params: 7,212,544 (27.51 MB)

In [23]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
# early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
checkpointer = [
    EarlyStopping(monitor='val_loss', verbose=1, restore_best_weights=True, mode="min", patience=10),
    ModelCheckpoint('best_model.keras', monitor="val_loss", verbose=1, save_best_only=True, mode="min")
]

In [ ]:
# Train the model
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    batch_size=64,
    epochs=50,
    verbose=1,
    callbacks=checkpointer
)


In [26]:
from tensorflow.keras.models import load_model

model = load_model('/content/best_model.keras')

In [ ]:
loss = model.evaluate(x_test,y_test)
print("Test Loss:", loss[0])
print("Test Accuracy:", loss[1])

In [ ]:
preds = model.predict(x_test)
y_pred = np.argmax(preds , axis = 1 )

In [ ]:
label_dict = {0 : 'anger', 1 : 'disgust', 2 : 'fear', 3 : 'sad', 4 : 'neutral'}

figure = plt.figure(figsize=(20, 8))
for i, index in enumerate(np.random.choice(x_test.shape[0], size=24, replace=False)):
    ax = figure.add_subplot(4, 6, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(x_test[index]))
    predict_index = label_dict[(y_pred[index])]
    true_index = label_dict[np.argmax(y_test,axis=1)[index]]

    ax.set_title("{} ({})".format((predict_index),
                                  (true_index)),
                                  color=("green" if predict_index == true_index else "red"))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_true = np.argmax(y_test , axis = 1 )
cm = confusion_matrix(y_true, y_pred)

# Plot the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()
